In [ ]:
# 1) quasi random numbers
# 2) importance sampling: sample from the region of the tail (look for extremes) that is most important, this will reduce variance 

BASELINE

In [2]:
import numpy as np, pandas as pd, scipy.stats as st

In [3]:
data = pd.read_csv('data.csv')
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
data['sec_loading'], data['t'] = r[data['sector'].values], st.norm.ppf(data.p)
factors, sample = np.random.normal(0,1, (100_000, len(r)+len(data))), []

In [4]:
data

,sector,m,d,p,sec_loading,t
0,5,-7.417203,0.631652,0.015994,0.64,-2.144549
1,2,-527.362999,49.292996,0.005138,0.41,-2.566434
2,2,-439.116574,42.366041,0.005386,0.41,-2.550002
3,5,112.839687,13.193781,0.016846,0.64,-2.123749
4,5,93.661362,14.562288,0.017555,0.64,-2.107093
...,...,...,...,...,...,...
5653,5,81.071783,3.121474,0.014192,0.64,-2.191924
5654,5,-29.621411,4.334980,0.015017,0.64,-2.169647
5655,5,71.006290,8.699961,0.014092,0.64,-2.194725
5656,5,102.410688,14.274780,0.017539,0.64,-2.107466


In [5]:
for obs in factors:
    m_factor, sec_factor, res_factor = obs[0], obs[:len(r)][data.sector.values], obs[len(r):]
    ind = r[0]**.5 * m_factor + (data.sec_loading-r[0])**.5 * sec_factor + (1-
                                                                        data.sec_loading)**.5 * res_factor < data.t
    loss = np.zeros((len(data),))
    loss[ind] = data[ind].m + data[ind].d * np.random.standard_t(3, size=sum(ind
                                                                         ))
    sample.append(sum(loss))

In [6]:
VaR = sorted([-s for s in sample])[100]

In [7]:
VaR 

-17095.554319596158